In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [2]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [3]:
train_ds='../data/train/labeled_data_processed.csv'
# test_ds='../data/test_data.csv'

train_df=pd.read_csv(train_ds)[['cuted_content','label']]
train_df

,cuted_content,label
0,人民网 北京 8 月 31 日电 孙竞 日前 进一步 规范 高等学校 命名 工作 依据 相关...,2
1,一年 落叶 黄 一阵 秋雨 一阵 凉 整日 奔波 工作 忙 出门 别忘添 衣裳 金秋 时节 ...,0
2,作者 祝 安顺 深圳大学 饶宗 颐 文化 研究院 编审 副教授 越来越 名校 毕业生 入职 ...,2
3,此前 其有 品牌 Madawell 合作 系列 这回 确认 推出 名下 服装 系列 采访 中...,3
4,眼下 疫情 依然 严峻 我国 大中小学 都 已 陆续 开学 疫情 防控 常态 化 学生 校园...,2
...,...,...
6995,近期 国家 卫健委 发布 学校 传染病 症状 监测 预警 技术 指南 学校 传染病 达 预警...,2
6996,新冠 肺炎 疫情 影响 之下 房企 纷纷 开设 线上 售楼处 据克而瑞 监测 超过 150 ...,1
6997,学生 犯错 后 教师 惩戒 惩戒 惩戒 边界 9 月 广东省 学校 条例 简称 条例 施行 ...,2
6998,6 月 3 日 下午 荣耀 带来 全新 荣耀 Play4 系列 新品 包括 Play4 Pl...,5


In [4]:
# 向量化文本
MAX_NUM_WORDS = 100000 # 最多保留不同词语词数，频率从大到小
MAX_SEQUENCE_LEN = 256
EMBEDDING_DIM=128

def tokenize(lang,max_num_words=None,max_sequence_len=300):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_num_words, 
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~', lower=True) # 
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,maxlen=max_sequence_len,
                                                            padding='post',truncating='post') # NOTE
    print('* Total different words: %s.' % len(lang_tokenizer.word_index))
    return tensor, lang_tokenizer
# get X
X,_=tokenize(train_df['cuted_content'].values,MAX_NUM_WORDS,MAX_SEQUENCE_LEN)
print(X.shape)

* Total different words: 143800.
(7000, 256)


In [5]:
# get Y one-hot
Y=pd.get_dummies(train_df['label']).values
print(Y.shape)

(7000, 7)


In [6]:
# split train and val
X_train=X
Y_train=Y
print(X_train.shape,Y_train.shape)

# X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size = 0.10)
# print(X_val.shape,Y_val.shape)

(6300, 256) (6300, 7)
(700, 256) (700, 7)


In [7]:
# build model
DROUPOUT=0.2
RECURRENT_DROPOUT=0.2
UNITS=128

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(tf.keras.layers.SpatialDropout1D(DROUPOUT))
model.add(tf.keras.layers.LSTM(UNITS, dropout=DROUPOUT, recurrent_dropout=RECURRENT_DROPOUT))
model.add(tf.keras.layers.Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 256, 128)          12800000  
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 256, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 7)                 903       
Total params: 12,932,487
Trainable params: 12,932,487
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
# callbacks
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)

callbacks=[]
# callbacks.append(early_stopping)

In [ ]:
# train model
EPOCHS=10
BATCH_SIZE=64

VAL_SPLIT=0.1

history = model.fit(X_train, Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,validation_split=VAL_SPLIT,
                    callbacks=callbacks)

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='val')
plt.legend()
plt.show()